In [7]:
from sqlite3 import Date
import pandas as pd
import yfinance as yf
import datetime as datetime
from prophet import Prophet
from time import strftime
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

In [11]:
ticker = 'AAPL'
today = datetime.date.today()
end_date = today.strftime("%Y-%m-%d")
data = yf.Ticker(ticker).history(period='max', interval='1d')
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1980-12-12 00:00:00-05:00,0.099722,0.100155,0.099722,0.099722,469033600,0.0,0.0
1980-12-15 00:00:00-05:00,0.094953,0.094953,0.094519,0.094519,175884800,0.0,0.0
1980-12-16 00:00:00-05:00,0.088015,0.088015,0.087582,0.087582,105728000,0.0,0.0
1980-12-17 00:00:00-05:00,0.089749,0.090183,0.089749,0.089749,86441600,0.0,0.0
1980-12-18 00:00:00-05:00,0.092351,0.092785,0.092351,0.092351,73449600,0.0,0.0
...,...,...,...,...,...,...,...
2023-02-13 00:00:00-05:00,150.949997,154.259995,150.919998,153.850006,62199000,0.0,0.0
2023-02-14 00:00:00-05:00,152.119995,153.770004,150.860001,153.199997,61707600,0.0,0.0
2023-02-15 00:00:00-05:00,153.110001,155.500000,152.880005,155.330002,65669300,0.0,0.0


In [12]:
df_forecast = data.copy()
df_forecast.reset_index(inplace=True)
df_forecast

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,1980-12-12 00:00:00-05:00,0.099722,0.100155,0.099722,0.099722,469033600,0.0,0.0
1,1980-12-15 00:00:00-05:00,0.094953,0.094953,0.094519,0.094519,175884800,0.0,0.0
2,1980-12-16 00:00:00-05:00,0.088015,0.088015,0.087582,0.087582,105728000,0.0,0.0
3,1980-12-17 00:00:00-05:00,0.089749,0.090183,0.089749,0.089749,86441600,0.0,0.0
4,1980-12-18 00:00:00-05:00,0.092351,0.092785,0.092351,0.092351,73449600,0.0,0.0
...,...,...,...,...,...,...,...,...
10631,2023-02-13 00:00:00-05:00,150.949997,154.259995,150.919998,153.850006,62199000,0.0,0.0
10632,2023-02-14 00:00:00-05:00,152.119995,153.770004,150.860001,153.199997,61707600,0.0,0.0
10633,2023-02-15 00:00:00-05:00,153.110001,155.500000,152.880005,155.330002,65669300,0.0,0.0
10634,2023-02-16 00:00:00-05:00,153.509995,156.330002,153.350006,153.710007,68167900,0.0,0.0


In [15]:
df_forecast["ds"] = df_forecast['Date'].dt.date
df_forecast["y"] = df_forecast["Close"]
df_forecast = df_forecast[["ds","y"]]
df_forecast

,ds,y
0,1980-12-12,0.099722
1,1980-12-15,0.094519
2,1980-12-16,0.087582
3,1980-12-17,0.089749
4,1980-12-18,0.092351
...,...,...
10631,2023-02-13,153.850006
10632,2023-02-14,153.199997
10633,2023-02-15,155.330002
10634,2023-02-16,153.710007


In [16]:
model = Prophet()
model.fit(df_forecast)

11:27:31 - cmdstanpy - INFO - Chain [1] start processing
11:27:39 - cmdstanpy - INFO - Chain [1] done processing


In [34]:
future = pd.to_datetime(end_date) + pd.DateOffset(days=30)
future_date = future.strftime("%Y-%m-%d")
dates = pd.date_range(start= end_date, end= future_date)
df_pred = pd.DataFrame({"ds" : dates})
forecast = model.predict(df_pred)
prediction_list = forecast.tail(30)
prediction_list

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
1,2023-02-20,146.037123,133.952394,159.107351,146.037123,146.037123,0.258312,0.258312,0.258312,0.015359,0.015359,0.015359,0.242953,0.242953,0.242953,0.0,0.0,0.0,146.295435
2,2023-02-21,146.085014,135.032792,158.689251,146.085014,146.085014,0.219679,0.219679,0.219679,0.038463,0.038463,0.038463,0.181216,0.181216,0.181216,0.0,0.0,0.0,146.304693
3,2023-02-22,146.132905,135.077226,159.034361,146.132905,146.132905,0.164214,0.164214,0.164214,0.047248,0.047248,0.047248,0.116966,0.116966,0.116966,0.0,0.0,0.0,146.297119
4,2023-02-23,146.180795,133.712317,158.928892,146.180795,146.180795,0.039547,0.039547,0.039547,-0.011565,-0.011565,-0.011565,0.051112,0.051112,0.051112,0.0,0.0,0.0,146.220342
5,2023-02-24,146.228686,133.877695,158.093575,146.228686,146.228686,-0.095002,-0.095002,-0.095002,-0.079580,-0.079580,-0.079580,-0.015422,-0.015422,-0.015422,0.0,0.0,0.0,146.133684
6,2023-02-25,146.276577,134.206604,158.921004,146.276577,146.276577,-0.086673,-0.086673,-0.086673,-0.004963,-0.004963,-0.004963,-0.081710,-0.081710,-0.081710,0.0,0.0,0.0,146.189904
7,2023-02-26,146.324468,133.802896,159.088425,146.324468,146.324468,-0.151812,-0.151812,-0.151812,-0.004963,-0.004963,-0.004963,-0.146849,-0.146849,-0.146849,0.0,0.0,0.0,146.172655
8,2023-02-27,146.372358,133.580697,158.220500,146.372358,146.372358,-0.194616,-0.194616,-0.194616,0.015359,0.015359,0.015359,-0.209975,-0.209975,-0.209975,0.0,0.0,0.0,146.177742
9,2023-02-28,146.420249,134.169915,158.165597,146.420249,146.420249,-0.231816,-0.231816,-0.231816,0.038463,0.038463,0.038463,-0.270279,-0.270279,-0.270279,0.0,0.0,0.0,146.188433
10,2023-03-01,146.468140,134.607380,158.610445,146.468140,146.468140,-0.279776,-0.279776,-0.279776,0.047248,0.047248,0.047248,-0.327024,-0.327024,-0.327024,0.0,0.0,0.0,146.188364


In [35]:
forecast = prediction_list[['ds','trend']]
forecast

,ds,trend
1,2023-02-20,146.037123
2,2023-02-21,146.085014
3,2023-02-22,146.132905
4,2023-02-23,146.180795
5,2023-02-24,146.228686
6,2023-02-25,146.276577
7,2023-02-26,146.324468
8,2023-02-27,146.372358
9,2023-02-28,146.420249
10,2023-03-01,146.468140


In [37]:
print(forecast.to_json(orient='records'))

[{"ds":1676851200000,"trend":146.0371230717},{"ds":1676937600000,"trend":146.0850138186},{"ds":1677024000000,"trend":146.1329045654},{"ds":1677110400000,"trend":146.1807953122},{"ds":1677196800000,"trend":146.2286860591},{"ds":1677283200000,"trend":146.2765768059},{"ds":1677369600000,"trend":146.3244675527},{"ds":1677456000000,"trend":146.3723582995},{"ds":1677542400000,"trend":146.4202490464},{"ds":1677628800000,"trend":146.4681397932},{"ds":1677715200000,"trend":146.51603054},{"ds":1677801600000,"trend":146.5639212869},{"ds":1677888000000,"trend":146.6118120337},{"ds":1677974400000,"trend":146.6597027805},{"ds":1678060800000,"trend":146.7075935274},{"ds":1678147200000,"trend":146.7554842742},{"ds":1678233600000,"trend":146.803375021},{"ds":1678320000000,"trend":146.8512657679},{"ds":1678406400000,"trend":146.8991565147},{"ds":1678492800000,"trend":146.9470472615},{"ds":1678579200000,"trend":146.9949380084},{"ds":1678665600000,"trend":147.0428287552},{"ds":1678752000000,"trend":147.09

In [38]:
print(forecast.to_json(orient='index'))

{"1":{"ds":1676851200000,"trend":146.0371230717},"2":{"ds":1676937600000,"trend":146.0850138186},"3":{"ds":1677024000000,"trend":146.1329045654},"4":{"ds":1677110400000,"trend":146.1807953122},"5":{"ds":1677196800000,"trend":146.2286860591},"6":{"ds":1677283200000,"trend":146.2765768059},"7":{"ds":1677369600000,"trend":146.3244675527},"8":{"ds":1677456000000,"trend":146.3723582995},"9":{"ds":1677542400000,"trend":146.4202490464},"10":{"ds":1677628800000,"trend":146.4681397932},"11":{"ds":1677715200000,"trend":146.51603054},"12":{"ds":1677801600000,"trend":146.5639212869},"13":{"ds":1677888000000,"trend":146.6118120337},"14":{"ds":1677974400000,"trend":146.6597027805},"15":{"ds":1678060800000,"trend":146.7075935274},"16":{"ds":1678147200000,"trend":146.7554842742},"17":{"ds":1678233600000,"trend":146.803375021},"18":{"ds":1678320000000,"trend":146.8512657679},"19":{"ds":1678406400000,"trend":146.8991565147},"20":{"ds":1678492800000,"trend":146.9470472615},"21":{"ds":1678579200000,"trend